In [1]:
# Usual Libraries USED FROM THE OTHER NOTEBOOK WE ADD MORE OR DELETE WHEN NECESSESARRY 
import pandas as pd
import numpy as np
import distutils

import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

# Librosa (the mother of audio files)
import librosa
import librosa.display
import IPython.display as ipd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

# Define the context features
context_features = {
    'video_id': tf.io.FixedLenFeature([], tf.string),
    'start_time_seconds': tf.io.FixedLenFeature([], tf.float32),
    'end_time_seconds': tf.io.FixedLenFeature([], tf.float32),
    'labels': tf.io.VarLenFeature(tf.int64)
}

# Define the sequence features
sequence_features = {
    'audio_embedding': tf.io.VarLenFeature(tf.string)
}

def _parse_function(example_proto):
    # Parse the example into context and sequence features
    context_data, sequence_data = tf.io.parse_single_sequence_example(
        example_proto,
        context_features=context_features,
        sequence_features=sequence_features
    )
    
    # Decode the audio embeddings from the byte strings
    audio_embeddings = tf.io.decode_raw(sequence_data['audio_embedding'].values, tf.uint8)
    audio_embeddings = tf.reshape(audio_embeddings, [-1, 128])  # Reshape to [time_steps, embedding_size]
    
    # Normalize the embeddings to the range [0, 1]
    audio_embeddings = tf.cast(audio_embeddings, tf.float32) / 255.0
    
    # Flatten the embedding to a 1D array
    flattened_embedding = tf.reshape(audio_embeddings, [-1])
    
    # Convert sparse labels to dense
    labels = tf.sparse.to_dense(context_data['labels'])
    
    return flattened_embedding, labels

def parse_tfrecords_to_dataframe(tfrecord_files):
    # Initialize an empty list to store embeddings and labels
    embeddings_list = []
    labels_list = []

    # Iterate through each TFRecord file in the folder
    for tfrecord_file in tqdm(tfrecord_files, desc="Processing TFRecord files"):
        raw_dataset = tf.data.TFRecordDataset(tfrecord_file)
        parsed_dataset = raw_dataset.map(_parse_function)

        # Iterate over the parsed dataset and collect embeddings and labels
        for audio_embedding, labels in parsed_dataset:
            # Convert to numpy arrays
            audio_embedding_np = audio_embedding.numpy()
            labels_np = labels.numpy()

            # Flatten the label array if necessary
            labels_np = labels_np.flatten() if len(labels_np.shape) > 0 else labels_np

            # Store the embedding and label
            embeddings_list.append(audio_embedding_np)
            labels_list.append(labels_np)

    # Create a DataFrame from the collected data
    df = pd.DataFrame({
        'embedding': embeddings_list,
        'labels': labels_list
    })

    return df

# Specify the folder containing TFRecord files
tfrecord_folder = '.github\Data\\bal_train'
tfrecord_files = [os.path.join(tfrecord_folder, f) for f in os.listdir(tfrecord_folder) if f.endswith('.tfrecord')]

# Parse the TFRecords into a DataFrame
df = parse_tfrecords_to_dataframe(tfrecord_files)




FileNotFoundError: [Errno 2] No such file or directory: '.github\\Data\\bal_train'

In [8]:
df.shape
df.describe

<bound method NDFrame.describe of                                               embedding              labels
0     [0.37254903, 0.47843137, 0.5764706, 0.05490196...               [399]
1     [0.3137255, 0.70980394, 0.84705883, 0.6431373,...            [0, 451]
2     [0.41960785, 0.5882353, 0.34901962, 0.5411765,...           [27, 466]
3     [0.5529412, 0.3254902, 0.43137255, 0.69411767,...        [0, 95, 137]
4     [0.4, 0.18431373, 0.6666667, 0.23137255, 0.533...          [427, 431]
...                                                 ...                 ...
7966  [0.5647059, 0.6431373, 0.8039216, 0.4, 0.70980...             [0, 52]
7967  [0.26666668, 0.17254902, 0.8352941, 0.3882353,...             [0, 50]
7968  [0.34509805, 0.09803922, 0.654902, 0.39607844,...        [0, 26, 506]
7969  [0.6784314, 0.105882354, 0.49411765, 0.3725490...               [247]
7970  [0.7607843, 0.63529414, 0.9411765, 0.34117648,...  [27, 33, 137, 254]

[7971 rows x 2 columns]>

Reshape them  embeddings

In [ ]:
import numpy as np

# Parameters (adjust as necessary)
embedding_length = 128  # This should match the original dimension of each heatmap slice
sequence_length = 100  # Adjust to match the desired number of time steps per sequence

# Convert the embedding column to a numpy array and reshape each embedding
def reshape_embeddings(df, embedding_length, sequence_length):
    reshaped_embeddings = []
    for embedding in df['embedding']:
        # Ensure embedding is a numpy array
        embedding = np.array(embedding)
        
        # Reshape the embedding into (sequence_length, embedding_length)
        # If necessary, truncate or pad to the sequence_length
        if len(embedding) < sequence_length * embedding_length:
            # Pad with zeros if too short
            padding_length = sequence_length * embedding_length - len(embedding)
            embedding = np.pad(embedding, (0, padding_length), mode='constant')
        elif len(embedding) > sequence_length * embedding_length:
            # Truncate if too long
            embedding = embedding[:sequence_length * embedding_length]
        
        # Reshape to (sequence_length, embedding_length)
        embedding = embedding.reshape((sequence_length, embedding_length))
        reshaped_embeddings.append(embedding)

    # Convert to numpy array for model input
    reshaped_embeddings = np.array(reshaped_embeddings)
    return reshaped_embeddings

# Apply reshaping to embeddings
reshaped_embeddings = reshape_embeddings(df, embedding_length, sequence_length)

# Output the reshaped array's shape to verify
print("Reshaped Embeddings Shape:", reshaped_embeddings.shape)
